## 导入所需库

主要包括os、PyTorch、scikit-learn、numpy、pandas和tqdm。并检查是否有GPU可用并设置计算设备。
注意默认使用GPU0，需要更换计算设备调节 `cuda:0` 变量即可，cuda使用英伟达显卡计算，Mac系统并不支持。

In [5]:
import os

import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# 检查是否有GPU可用并指定设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(torch.cuda.is_available())

True


## 定义辅助函数

### 数据读取与处理函数

* 读取数据：从CSV文件中读取数据，并将第一列转换为时间索引。
* 过滤数据：根据提供的起止日期，过滤指定时间段的数据。
* 提取特征：提取目标变量Y、特征变量X和F，以及其他宏观变量M。
* 返回数据：返回处理后的数据，包括Y、X、F、M和时间索引。

In [6]:
def process_data(filename, start_date=None, end_date=None):
    # 读取数据
    df = pd.read_csv(filename)

    # 将第一列设置为时间索引并转换为日期类型
    df['Unnamed: 0'] = pd.to_datetime(df['Unnamed: 0'], format='%Y%m')
    df.set_index('Unnamed: 0', inplace=True)

    # 如果提供了起止日期，则截取该时间段的数据
    if start_date and end_date:
        df = df.loc[pd.to_datetime(start_date, format='%Y%m'):pd.to_datetime(end_date, format='%Y%m')]

    # 读取Y数组（以xr_开头的变量）
    Y = df.filter(regex='^xr_').values

    # 读取F数组（列名以m结尾的数据）
    F = df.filter(regex='m$').values

    # 读取X数组（以f_开头的变量）
    X = df.filter(regex='^f_').values

    # 读取M数组（其他变量）
    M = df.drop(columns=df.filter(regex='(^xr_)|(^f_)|m$').columns).values

    # 返回时间索引列
    time = df.index
    return Y, X, F, M, time

### 计算R2OOS的函数
* 计算条件均值预测：计算条件均值预测并滞后一期。
* 计算残差平方和和总平方和：计算残差平方和（SSres）和总平方和（SStot）。
* 计算R2OOS分数：返回R2OOS分数。
逻辑与原文中保持了一致，对于实际计算中这部分是否合理的讨论，见discussion.pdf。

In [7]:
def R2OOS(y_true, y_forecast):
    # 计算条件均值预测
    y_condmean = np.divide(y_true.cumsum(), (np.arange(y_true.size) + 1))

    # 滞后一期
    y_condmean = np.insert(y_condmean, 0, np.nan)
    y_condmean = y_condmean[:-1]
    y_condmean[np.isnan(y_forecast)] = np.nan

    # 计算残差平方和和总平方和
    SSres = np.nansum(np.square(y_true - y_forecast))
    SStot = np.nansum(np.square(y_true - y_condmean))

    return 1 - SSres / SStot

### 定义文件读写操作的函数

In [8]:
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_model_config(model, file_path):
    with open(file_path, 'w') as f:
        f.write(str(model))

## 定义全局变量
* 用于控制网络结构，定义模型结构参数，包括隐藏层数、隐藏节点数和输出节点数。
* 定义M的分组：根据不同的特征维度对M进行分组。

如果不需要进行分组M_GROUPS列表全设为1即可，如果不需要宏观变量，设置为空列表即可。

In [9]:
# 全局变量定义
HIDDEN_LAYERS_MODEL_ONE = 2
HIDDEN_NODES_MODEL_ONE = [128, 128]
OUTPUT_NODES_MODEL_ONE = 32

HIDDEN_LAYERS_MODEL_TWO = 3
HIDDEN_NODES_MODEL_TWO = [8, 4, 2]
OUTPUT_NODES_MODEL_TWO = 1

# M 分组的全局变量
M_GROUPS = [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5]

## 神经网络结构的定义

In [10]:
class Model_one(nn.Module):
    def __init__(self, input_size, hidden_sizes=[64], dropout_rate=0.5, output_nodes=OUTPUT_NODES_MODEL_ONE):
        super(Model_one, self).__init__()
        self.layers = nn.ModuleList()
        self.dropout = nn.Dropout(dropout_rate)

        prev_size = input_size
        for hidden_size in hidden_sizes:
            self.layers.append(nn.Linear(prev_size, hidden_size))
            prev_size = hidden_size

        self.norm_layer = nn.BatchNorm1d(hidden_sizes[-1])
        self.output_layer = nn.Linear(hidden_sizes[-1], output_nodes)

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
            x = self.dropout(x)
        x = self.norm_layer(x)
        x = self.output_layer(x)
        return x


class Model_two(nn.Module):
    def __init__(self, input_size, hidden_layers=HIDDEN_LAYERS_MODEL_TWO, dropout_rate=0.5,
                 hidden_nodes=HIDDEN_NODES_MODEL_TWO, output_nodes=OUTPUT_NODES_MODEL_TWO):
        super(Model_two, self).__init__()
        self.layers = nn.ModuleList()

        self.layers.append(nn.Sequential(
            nn.Linear(input_size, hidden_nodes[0]),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        ))

        for i in range(1, hidden_layers):
            self.layers.append(nn.Sequential(
                nn.Linear(hidden_nodes[i - 1], hidden_nodes[i]),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ))

        self.norm_layer = nn.BatchNorm1d(hidden_nodes[-1])
        self.output_layer = nn.Linear(hidden_nodes[-1], output_nodes)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = self.norm_layer(x)
        x = self.output_layer(x)
        return x


class IntegratedModel(nn.Module):
    def __init__(self, input_size_model_one, output_size, m_groups, dropout_rate=0.5):
        super(IntegratedModel, self).__init__()
        self.model_one = Model_one(input_size_model_one, dropout_rate=dropout_rate)

        # 创建一个字典，键是组号，值是每个组的Model_two实例
        self.model_twos = nn.ModuleDict({
            str(group): Model_two(sum(1 for g in m_groups if g == group), dropout_rate=dropout_rate)
            for group in set(m_groups)
        }) if m_groups else None

        # 调整最终输出层的输入尺寸
        final_input_size = OUTPUT_NODES_MODEL_ONE + (OUTPUT_NODES_MODEL_TWO * len(self.model_twos) if self.model_twos else 0)
        self.final_layer = nn.Linear(final_input_size, output_size)

    def forward(self, x1, x2, m_groups):
        out_one = self.model_one(x1)

        if self.model_twos:
            # 获取每个组的Model_two的输出
            outs_two = []
            for group in set(m_groups):
                group_indices = [i for i, g in enumerate(m_groups) if g == group]
                group_input = x2[:, group_indices]
                group_output = self.model_twos[str(group)](group_input)
                outs_two.append(group_output)

            combined_out_two = torch.cat(outs_two, dim=1)
            combined_out = torch.cat((out_one, combined_out_two), dim=1)
        else:
            combined_out = out_one

        final_out = self.final_layer(combined_out)
        return final_out

## 定义超参数搜索逻辑
超参数搜索：优化模型的dropout率和L2正则化系数。每次组合不同的dropout率和L2正则化系数（网络搜索），训练模型并评估验证损失，选择最优的超参数。

In [11]:
def hyperparameter_search(F_train, M_train, Y_train, dropout_rates, l2_regs, m_groups, batch_size=32):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    best_val_loss = np.inf
    best_params = None
    print("\n 参数寻优中，请稍后~")

    for dropout_rate in dropout_rates:
        for l2_reg in l2_regs:
            model = IntegratedModel(F_train.shape[1], M_train.shape[1] if m_groups else 0, Y_train.shape[1], m_groups, dropout_rate).to(device)
            optimizer = optim.AdamW(model.parameters(), weight_decay=l2_reg)
            loss_fn = nn.MSELoss()

            F_train_sub, F_val, M_train_sub, M_val, Y_train_sub, Y_val = train_test_split(
                F_train.cpu().numpy(), M_train.cpu().numpy(), Y_train.cpu().numpy(), test_size=0.15, random_state=3407)

            F_train_sub, F_val = torch.tensor(F_train_sub, dtype=torch.float32, device=device), torch.tensor(F_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device)
            M_train_sub, M_val = torch.tensor(M_train_sub, dtype=torch.float32, device=device), torch.tensor(M_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device) if m_groups else (None, None)
            Y_train_sub, Y_val = torch.tensor(Y_train_sub, dtype=torch.float32, device=device), torch.tensor(Y_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device)

            train_dataset = torch.utils.data.TensorDataset(F_train_sub, M_train_sub, Y_train_sub) if m_groups else torch.utils.data.TensorDataset(F_train_sub, Y_train_sub)
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

            early_stop_counter = 0
            min_val_loss = np.inf

            for epoch in tqdm(range(10000), desc=f"Optimizing: Dropout {dropout_rate}, L2 {l2_reg}"):
                model.train()
                for data in train_loader:
                    optimizer.zero_grad()
                    if m_groups:
                        F_batch, M_batch, Y_batch = data
                        outputs = model(F_batch, M_batch, m_groups)
                    else:
                        F_batch, Y_batch = data
                        outputs = model(F_batch, None, m_groups)
                    loss = loss_fn(outputs, Y_batch)
                    loss.backward()
                    optimizer.step()

                model.eval()
                with torch.no_grad():
                    if m_groups:
                        val_outputs = model(F_val, M_val, m_groups)
                    else:
                        val_outputs = model(F_val, None, m_groups)
                    val_loss = loss_fn(val_outputs, Y_val)

                if val_loss.item() < min_val_loss:
                    min_val_loss = val_loss.item()
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1

                if early_stop_counter > 500:
                    print(f"\n早停，以防过拟合。min_val_loss:{min_val_loss}")
                    break

            if min_val_loss < best_val_loss:
                best_val_loss = min_val_loss
                best_params = {'dropout_rate': dropout_rate, 'l2_reg': l2_reg}

    print(f"\n最佳超参数：{best_params}")
    return best_params

## 定义主函数
* 读取和处理数据：从CSV文件中读取数据，并进行特征缩放。
* 数据分割：将数据分割成训练集和测试集。
* 特征缩放：对特征进行归一化处理。
* 初始化预测数组：初始化一个全是NaN的数组，用于存储预测结果。
* 逐个处理测试样本：每隔48个样本进行一次超参数搜索，并训练多个模型，选择最优模型进行预测。
* 保存结果：将预测结果和实际值保存为CSV文件，并计算R2OOS分数。

关键参数在于`n_trials=20`, `k_best=2`, 用于控制重复训练几次和取前几个模型的平均值作为预测值。

In [12]:
def main(filename, start_date, end_date, split_date, m_groups, n_trials=20, k_best=2):
    base_dir = './jknetwork'
    results_dir = './jknetwork/results'
    ensure_dir(base_dir)
    ensure_dir(results_dir)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    Y, X, F, M, time = process_data(filename, start_date, end_date)

    # 如果想同时使用X和F特征，取消以下代码的注释
    # F = np.hstack((X, F))

    # 如果想使用X特征，取消以下代码的注释
    # F = X
    
    Y_tensor = torch.FloatTensor(Y).to(device)
    F_tensor = torch.FloatTensor(F).to(device)
    M_tensor = torch.FloatTensor(M).to(device)

    split_idx = np.where(time == pd.to_datetime(split_date, format='%Y%m'))[0][0]
    F_train, F_test = F_tensor[:split_idx], F_tensor[split_idx:]
    M_train, M_test = M_tensor[:split_idx], M_tensor[split_idx:]
    Y_train, Y_test = Y_tensor[:split_idx], Y_tensor[split_idx:]

    scaler_F = MinMaxScaler()
    F_train = torch.FloatTensor(scaler_F.fit_transform(F_train.cpu().numpy())).to(device)
    F_test = torch.FloatTensor(scaler_F.transform(F_test.cpu().numpy())).to(device)

    scaler_M = MinMaxScaler()
    M_train = torch.FloatTensor(scaler_M.fit_transform(M_train.cpu().numpy())).to(device) if m_groups else None
    M_test = torch.FloatTensor(scaler_M.transform(M_test.cpu().numpy())).to(device) if m_groups else None

    Ypre = np.full(Y.shape, np.nan)

    predictions = []

    for i in tqdm(range(len(F_test)), desc="Processing Test Samples"):
        time_point_dir = os.path.join(base_dir, f'time_point_{i}')
        ensure_dir(time_point_dir)
        if i % 48 == 0:
            dropout_rates = [0.1, 0.3, 0.5]
            l2_regs = [0.01, 0.004, 0.007, 0.001]
            best_params = hyperparameter_search(F_train, M_train if m_groups else np.zeros((len(F_train), 0)), Y_train, dropout_rates, l2_regs, m_groups, batch_size=32)
        trial_models = []
        for trial in range(n_trials):
            model = IntegratedModel(F_train.shape[1], M_train.shape[1] if m_groups else 0, Y_train.shape[1], m_groups, best_params['dropout_rate']).to(device)
            optimizer = optim.AdamW(model.parameters(), weight_decay=best_params['l2_reg'])
            loss_fn = nn.MSELoss()

            F_train_val, F_val, M_train_val, M_val, Y_train_val, Y_val = train_test_split(
                F_train.cpu().numpy(), M_train.cpu().numpy(), Y_train.cpu().numpy(), test_size=0.15, random_state=3407)
            F_train_val, F_val = torch.tensor(F_train_val, dtype=torch.float32, device=device), torch.tensor(F_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device)
            M_train_val, M_val = torch.tensor(M_train_val, dtype=torch.float32, device=device), torch.tensor(M_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device) if m_groups else (None, None)
            Y_train_val, Y_val = torch.tensor(Y_train_val, dtype=torch.float32, device=device), torch.tensor(Y_val,
                                                                                                             dtype=torch.float32,
                                                                                                             device=device)

            train_dataset = torch.utils.data.TensorDataset(F_train_val, M_train_val, Y_train_val) if m_groups else torch.utils.data.TensorDataset(F_train_val, Y_train_val)
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)

            min_val_loss = np.inf
            early_stop_counter = 0
            for epoch in range(10000):
                model.train()
                for data in train_loader:
                    optimizer.zero_grad()
                    if m_groups:
                        F_batch, M_batch, Y_batch = data
                        outputs = model(F_batch, M_batch, m_groups)
                    else:
                        F_batch, Y_batch = data
                        outputs = model(F_batch, None, m_groups)
                    loss = loss_fn(outputs, Y_batch)
                    loss.backward()
                    optimizer.step()

                model.eval()
                with torch.no_grad():
                    if m_groups:
                        val_outputs = model(F_val, M_val, m_groups)
                    else:
                        val_outputs = model(F_val, None, m_groups)
                    val_loss = loss_fn(val_outputs, Y_val)

                if val_loss.item() < min_val_loss:
                    min_val_loss = val_loss.item()
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1

                if early_stop_counter > 500:
                    print(f"Early stopping at trial {trial + 1}, epoch {epoch + 1} with val loss: {min_val_loss}")
                    break

            trial_models.append((min_val_loss, model))

        trial_models.sort(key=lambda x: x[0])
        best_models = trial_models[:k_best]

        model_preds = []
        for _, model in best_models:
            model.eval()
            with torch.no_grad():
                if m_groups:
                    test_pred = model(F_test[i:i + 1], M_test[i:i + 1], m_groups)
                else:
                    test_pred = model(F_test[i:i + 1], None, m_groups)
                model_preds.append(test_pred.cpu().numpy())

        avg_pred = np.mean(model_preds, axis=0)
        predictions.append(avg_pred.flatten().tolist())

        Ypre[split_idx + i] = avg_pred

        config_path = os.path.join(time_point_dir, 'network_config.txt')
        save_model_config(model, config_path)

        for rank, (loss, model) in enumerate(best_models):
            model_path = os.path.join(time_point_dir, f'best_model_{rank+1}.pt')
            torch.save(model.state_dict(), model_path)

    predictions = np.array(predictions)
    actuals = Y_test.cpu().numpy()
    results_df = pd.DataFrame({'Predictions': predictions.flatten(), 'Actuals': actuals.flatten()})
    results_df_path = os.path.join(results_dir, 'bigresults3.csv')
    results_df.to_csv(results_df_path, index=False)
    print("\n预测完成噜，结果已保存至", results_df_path)

    R2OOS_scores_per_dimension = [R2OOS(Y[:, i], Ypre[:, i]) for i in range(predictions.shape[1])]
    print("R2OOS Scores Per Dimension:", R2OOS_scores_per_dimension)


## 一切准备就绪，准备开启炼丹炉！

In [13]:
main('./data.csv', '197108', '202112', '199001', M_GROUPS)

Processing Test Samples:   0%|          | 0/384 [00:00<?, ?it/s]


 参数寻优中，请稍后~



Processing Test Samples:   0%|          | 0/384 [00:21<?, ?it/s]

KeyboardInterrupt

